In [2]:
import pandas as pd
import zipfile
import os

As the world becomes more and more interconnected, air travel becomes an ever more important means of transit for the United States. Despite this importance, U.S. flight delays have become more numerous and unpredictable, exacerbated by the recent pandemics, supply chain disruptions, and climate change. In order to help guide America's future aviation, it is essential that we be able to monitor and predict flight delays. And in order to do this, scienctists and statisticians need access to a large, high quality collection of relevant data.

Hoping to address this need, the Bureau of Transportation Statistics, or BTS, has maintained an up-to-date dataset of information on every flight in America that it believes will help researchers and professionals track and predict flight delays. This dataset of [Carrier On-Time Performance](https://www.transtats.bts.gov/tables.asp?qo_vq=EFD&QO_anzr=) contains data dating back to 1987 and aims to document all the information that could relevant to those seeking to model flight delays. However, the BTS is always seeking to improve its data collection. A cursory look through this database will reveal the fact that information for each flight is not always collected in a complete state and numerous additional columns of data have been added over the years.

In order to maximize the efficiency of its data collection, the BTS aims to prioritize the most important data columns to ensure that future models are as accurate as possible.

The purpose of this project is to create a predictive model of U.S. flight delays with the BTS dataset and identify the data columns that are most important to the model's predictions to help guide the BTS' future data collection efforts and create the best models of American flight delays possible.

## Data Collection

In order to accompilish this, we will using a dataset comprised of the BTS' flight delay records from November 2021 to October 2022. This span of time allows a full year's worth of up-to-date information to be used while minimizing the size of the dataset that will need to be processed by our model (Due to computer memory limitations) and avoiding training our model on the anomalous aviation time period that occured at the beginning of the Covid-19 pandemic.

Due to the fact that the BTS only provides data in one month increments, we will need to begin by taking the 12 files of data from each month and combining them into a single dataset for use in our modelling efforts.

The following two code cells automatically unpack our zipped folders of raw data and combine them into a single DataFrame.

In [3]:
def combine_zipped_data(root_data_folder_path):
    '''
    This function opens all zip files in a given folder and combines any csv data found
    within them into a single Pandas DataFrame.
    
    Parameters
    ----------
    root_data_folder_path: A string containing the path to a folder containing zip files
                           with csv data.
    
    Returns
    -------
    A Pandas Dataframe with all csv data found in the given folder, combined together along
    the index (0) axis.
    '''
    # Creating empty list of DataFrames
    data_to_combine = []
    # Looping through raw data folder
    with os.scandir(root_data_folder_path) as root_data_folder:
        total_files = len(os.listdir(root_data_folder_path))
        current_progress = 0
        for entry in root_data_folder:
            # Displaying current progress
            current_progress += 1
            print(f"Processing file {current_progress}/{total_files} ...", end="\r")
            # Searching for zipped data
            if entry.name.endswith(".zip") and entry.is_file():
                # Opening zipped data folders
                with zipfile.ZipFile(root_data_folder_path + '/' + entry.name, "r") as zipped:
                    for name in zipped.namelist():
                        # Searching for csv files in zipped folders
                        if name.endswith('.csv'):
                            with zipped.open(name) as delay_data:
                                # Reading csv and adding to list of datasets
                                data_to_combine.append(pd.read_csv(delay_data, low_memory=False))
    print('All files unpacked. Combining data...', end='\r')
    combined_data = pd.concat(data_to_combine)
    print('Data successfully combined!           ')
    return combined_data

In [4]:
delay_df = combine_zipped_data('./raw_data')

Data successfully combined!           


Before we can begin training a model to predict flight delays using the BTS's data, we must first ensure that the dataset is in good condition. Datasets frequently have missing values, unexpected data types, or erroneous entries that can prevent a model from being trained on it or adversely affect the model's predictive ability. Given the immense scale of the BTS's data, we should expect some of it to contain these issues.

In order to combat this, the dataset should be exhaustively cleaned before we attempt to use it.

### Cleaning Missing Values

Missing values, or `NaN`s, are placeholder values in a dataset that typically indicate the absence of data. It is very common for larger or more informal datasets to contain a significant quantity of `NaN`s. This poses an issue for our modelling efforts, as the vast majority of models cannot interpret `NaN` values in their training data. Consequently, the presence of any `NaN` values in our training dataset would prevent us from using it to train a flight delay predictor. As such, it is essential that we remove or replace these `NaN` values before continuing.

In order to determine how best to proceed, we can begin by inspecting the dataset to see how many `NaN` values are in each column.

In [5]:
# Displaying the nummber of NaNs in a small subset of the data's columns
delay_df.isna().sum()

Year                      0
Quarter                   0
Month                     0
DayofMonth                0
DayOfWeek                 0
                     ...   
Div5TotalGTime      6725064
Div5LongestGTime    6725064
Div5WheelsOff       6725064
Div5TailNum         6725064
Unnamed: 109        6725064
Length: 110, dtype: int64

In this small sample of columns from the dataset, we can see that some columns contain a monstrously large number number of `NaN` values. To understand just how much of each column is a `NaN` value, we can compare it to the number of rows in the dataset, shown by the cell below.

In [6]:
# Displaying the number of rows in the dataset for comparison to the number of NaNs
delay_df.shape[0]

6725064

In comparing the above number to the `NaN`s in the columns shown previously, we can see that _all_ the values in some columns are `NaN`.

This is because some of the columns contain information about flights that were diverted, with information for up to five diversions. Since the vast majority of flights are not diverted even once, let alone five times, these columns are almost always not applicable and thus contain `NaN` values.

When faced with `NaN` values, there are three general approaches that can be taken. If the column of data is not vital to our modelling purposes, then we can simply drop columns with `NaN`s. Similarly, if `NaN` values are randomly distributed throughout the rows of data, then we can remove rows of data that contain `NaN` values without misrepresenting the distribution that the dataset was pulled from. And lastly, we can attempt to replace, or impute, these missing values with our best estimate of the values that should be there.

Looking at our dataset, we can see that the the columns that have many `NaN`s are not only nearly devoid of data, they also contain information that is not relevant to our attempts to preemptively predict flight delays, such as information on diverted flights. For that reason, we can safely drop the columns that contain a large proportion of `NaN` values. This is accomplished in the following cell.

In [20]:
# Removing all columns that are more than 5% NaN values 
delay_df = delay_df[delay_df.columns[delay_df.isna().sum() < delay_df.shape[0] / 20]]

The columns with smaller amounts of `NaN` values however, cannot be so easily discarded. A quick inspection of the remaining columns that contain `NaN`s reveals why.

In [21]:
# Inspecting columns with less than 5% NaNs
delay_df.columns[delay_df.isna().sum() > 0]

Index(['Tail_Number', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
       'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
       'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups', 'ActualElapsedTime', 'AirTime'],
      dtype='object')

In this list of columns with `NaN`s, we can see the column `ArrDel15` which indicates whether or not a flight's arrival is officially considered delayed by the BTS. This is our delay prediction model's target variable. This is the value it is attempting to predict.

Supervised machine learning, like the type we must perform to create a flight delay predictor, relies on the presence of labels that the model can train on in order to determine what each data point in its training data is. Data that does not contain a label value therefore cannot be used for model training.

To resolve this issue, we have the option of either dropping any rows that contain these values or replacing them with an estimate such as the median or mode of the dataset's labels. Dropping rows will introduce inaccuracies into our training data if the rows that are removed are not similar to the rows of data that were kept. Replacing these values can also similarly misrepresent the data if the labels do now follows the simple estimator that we use to do the replacing.

In order to decide how best to approach this, we can inspect the data with missing labels to see if it greatly differs from the remainder of the dataset.

We can begin by inspecting the proportion of flights from each state that have `NaN` delay values.

In [22]:
# Inspecting the proportion of flights by origin state that had NaN delay values
(delay_df[delay_df.ArrDel15.isna()].OriginStateName.value_counts() /\
delay_df.OriginStateName.value_counts()).sort_values(ascending=False)

Delaware                                          0.093750
U.S. Virgin Islands                               0.061710
Maine                                             0.051497
New Jersey                                        0.049989
New York                                          0.043800
Vermont                                           0.042594
Rhode Island                                      0.040019
North Dakota                                      0.039498
New Hampshire                                     0.037846
Virginia                                          0.035908
Connecticut                                       0.035568
Massachusetts                                     0.035545
Alaska                                            0.034058
West Virginia                                     0.033224
Wyoming                                           0.033012
Florida                                           0.032415
Ohio                                              0.0321

Here we can see that there is indeed a difference in the degree to which each state's flights have missing values. However, the vast majority of these values are of the same order of magnitude. This, coupled with the fact that only 2% of rows have missing values suggests that it may be safer to remove these rows outright than to impute them.

We can further inspect this fact by looking at the proportion of each month's flights that were missing delay information.

In [23]:
# Inspecting the proportion of flights by origin state that had NaN delay values
(delay_df[delay_df.ArrDel15.isna()].Month.value_counts() /\
delay_df.Month.value_counts()).sort_values(ascending=False)

1     0.063902
2     0.045524
6     0.033831
8     0.027451
12    0.026640
4     0.025849
5     0.022225
7     0.020855
3     0.017957
9     0.017518
10    0.010126
11    0.007594
Name: Month, dtype: float64

Here too we can see a difference in missing values by month, but never more than an order of magnitude's worth of difference.

When considering the fact that the missing data points are not extremely different from the rest of the dataset and that they make up only 2% of the data's rows, we can conclude that removing these rows would have a neglegible effect on how well the dataset represents the real world.

As such, we can safely drop them in the cell below.

In [24]:
# Dropping rows of data with NaN in the target delay column (ArrDel15)
delay_df = delay_df.dropna(subset="ArrDel15")

With the rows that contained `NaN` values for delay now addressed we can inspect the dataset once more to see if there are any `NaN`s remaining.

In [25]:
# Displaying the total number of NaNs in the trimmed dataset
delay_df.isna().sum().sum()

0

In doing so, we can confirm that our dataset has been completely cleaning of missing values and move on to correcting our dataset's data types.

### Cleaning Data Types

Now that our dataset has been cleaned of meaningless NaN values, we must inspect the columns of our data to ensure that they are stored as the correct data type. Incorrect data types can cause our flight delay model to interpret numeric data as words, interpret dates as text, or simply fail to interpret data altogether.

In [26]:
# Listing data types for each column
delay_df.dtypes

Year                                 int64
Quarter                              int64
Month                                int64
DayofMonth                           int64
DayOfWeek                            int64
FlightDate                          object
Reporting_Airline                   object
DOT_ID_Reporting_Airline             int64
IATA_CODE_Reporting_Airline         object
Tail_Number                         object
Flight_Number_Reporting_Airline      int64
OriginAirportID                      int64
OriginAirportSeqID                   int64
OriginCityMarketID                   int64
Origin                              object
OriginCityName                      object
OriginState                         object
OriginStateFips                      int64
OriginStateName                     object
OriginWac                            int64
DestAirportID                        int64
DestAirportSeqID                     int64
DestCityMarketID                     int64
Dest       

When comparing these data types to the data dictionary provided by the BTS [here](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ), we can see that all but one of these columns contain the correct data type.

While the vast majority of our data is correct, the `FlightDate` column is being represented as a string of text rather than as a date. In order to better represent the data in that column and prepare it for conversion to a direct numeric format if it is chosen for use in model training, this column should be converted to the datetime data type.

This is accomplished in the following cell.

In [27]:
# Converting flight date strings to datetime format
delay_df.FlightDate = pd.to_datetime(delay_df.FlightDate, format="%Y-%m-%d")

### Saving Cleaned Data


With our dataset fully collected and cleaned, it is now ready to be saved and used for EDA and pre-processing.

In [28]:
delay_df.to_pickle('./data/cleaned_aviation_data')